In [10]:
date='0704'

WIN PROBABILITY DATA

In [ ]:
import pandas as pd
import requests
from audl.stats.endpoints.gameevents import *
import numpy as np

features = ['thrower_x', 'thrower_y', 'start_on_offense', 'point_start_time', 'possession_num', 'possession_throw', 'game_quarter', 'quarter_point', 'is_home_team', 'home_team_score', 'away_team_score', 'gameID', 'total_points', 'home_teamID', 'away_teamID']

def process_games(GAMES):
    GAMES['total_points'] = GAMES['home_team_score'] + GAMES['away_team_score']
    GAMES = GAMES[GAMES.game_quarter < 5] ##TODO include overtimes
    GAMES['point_start_time'] = 720 - GAMES['point_start_time']
    GAMES = GAMES[features]

    games = []
    for gameID in GAMES.gameID.unique():
        GAME = GAMES[GAMES.gameID == gameID]
        points = []
        prev_point_time = 720
        current_quarter = 1
        prev_df = None
        error_points = 0
        for group_keys, group_df in GAME.groupby(['total_points', 'game_quarter']):
            if prev_df is None:
                prev_df = group_df
                continue

            prev_point_time = prev_df.point_start_time.max()
            if (current_quarter != group_df.game_quarter.max()):
                current_quarter = group_df.game_quarter.max()
                current_point_time = 0
            else:
                current_point_time = group_df.point_start_time.max()
            if current_point_time >= prev_point_time:
                error_points = error_points + 1
                continue
            times = np.linspace(prev_point_time, current_point_time + 1, len(prev_df))
            prev_df['times'] = times
            points.append(prev_df)
            prev_df = group_df
            if group_keys == (GAME.home_team_score.max()+GAME.away_team_score.max(), 4):
                times = np.linspace(current_point_time, 0, len(prev_df))
                prev_df['times'] = times
                points.append(prev_df)
        GAME_POINTS = pd.concat(points)
        GAME_POINTS.times = GAME_POINTS.times + ((4 - GAME_POINTS.game_quarter)*720)
        try:
            # if error_points > 0:
            #     print(f'{error_points} errors with {GAME_POINTS.gameID.iloc[0]}')
            assert GAME_POINTS.times.is_monotonic_decreasing, f'timing is off somewhere for game: {GAME_POINTS.gameID.iloc[0]}'
            home_team_win = GAME_POINTS[['away_team_score', 'home_team_score']].max().values.argmax() if GAME_POINTS['home_team_score'].max() != GAME_POINTS['away_team_score'].max() else -2
            GAME_POINTS['home_team_win'] = home_team_win
            games.append(GAME_POINTS)
        except AssertionError as e:
            print(e)

        PROCESSED_GAMES = pd.concat(games).drop(['point_start_time', 'start_on_offense'], axis=1)
        PROCESSED_GAMES['score_diff'] = PROCESSED_GAMES['home_team_score'] - PROCESSED_GAMES['away_team_score']
    return PROCESSED_GAMES[PROCESSED_GAMES.home_team_win >= 0]

base_url = "https://www.backend.audlstats.com/api/v1/"

endpoint = "games?date=2021:"
ids = pd.DataFrame(requests.get(f'{base_url}{endpoint}').json()['data'])['gameID'].values

game_events_proxy = GameEventsProxy()
all_games = []
for id in ids:
    try:
        game_events = GameEvents(id, game_events_proxy)
        game_events.process_game_events()
        game_df = game_events.get_events_df(True, True, True)
        all_games.append(game_df)
    except AssertionError as e:
        print(id)
        print(e)
    except Exception as e:
        print(id)
        raise e
ALL_GAMES = pd.concat(all_games)
ALL_GAMES.to_csv(f'../data/raw/all_games_{date}.csv', index=False)
df = process_games(ALL_GAMES)
df.to_csv(f'../data/processed/all_games_{date}.csv', index=False)


PLAYER DATA

In [12]:
from audl.stats.endpoints.playerstats import PlayerStats
import pandas as pd
year = 2021
dfs = []
while True:
    df = PlayerStats(year, 'total', 'all').fetch_table()
    df['year'] = year
    dfs.append(df)
    year = year+1
    if len(df) == 0:
        break
player_stats_by_year = pd.concat(dfs)
player_stats_by_year.to_csv(f'../data/raw/player_stat_by_year_{date}')

In [85]:
from audl.stats.endpoints.players import Players
from audl.stats.endpoints.teams import Teams

teams = Teams()
teams_df = teams.get_teams()
teams_df = pd.merge(teams_df.drop('division', axis=1).reset_index(drop=True), teams_df.division.apply(pd.Series).reset_index(drop=True), left_index=True, right_index=True)
teams_df.to_csv(f'../data/raw/teams_{date}')

players = Players()
players_df = players.get_players()
players_df = players_df.explode('teams')
players_df = pd.merge(players_df.drop('teams', axis=1).reset_index(drop=True), players_df.teams.apply(pd.Series).reset_index(drop=True), left_index=True, right_index=True)
players_df['firstName'] = players_df['firstName'].apply(lambda x:str(x).strip())
players_df['lastName'] = players_df['lastName'].apply(lambda x:str(x).strip())
players_df.to_csv(f'../data/raw/players_{date}')

In [86]:
len(players_df[players_df.lastName=='Grovic'].firstName.iloc[0])

5